In [1]:
%pip install -Uq penman

Note: you may need to restart the kernel to use updated packages.


In [2]:
import penman
import pandas as pd
from tqdm import tqdm
from functools import partial

tqdm.pandas()

In [3]:
news_df = pd.read_csv("../input/mind-news-dataset/MINDsmall_train/news.tsv", sep="\t", header=None, index_col=0, names=["category", "subcategory", "title", "abstract", "link", "title_entities", "abstract_entities"])
news_df

,category,subcategory,title,abstract,link,title_entities,abstract_entities
N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
...,...,...,...,...,...,...,...
N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...,https://assets.msn.com/labs/mind/BBWzQJK.html,"[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid...","[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid..."
N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN,https://assets.msn.com/labs/mind/BBWzQYV.html,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...",[]
N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]


In [4]:
len(news_df.dropna(subset="title")), len(news_df.dropna(subset="abstract"))

(51282, 48616)

In [5]:
labels = pd.read_csv("../input/mind-frames/framelabels.csv", index_col=0)
dimensions = pd.read_csv("../input/mind-frames/framedimensions.csv", index_col=0, header=[0,1])
structures = pd.read_csv("../input/mind-frames/framestructure.csv", index_col=0, header=None)

## # Extract Frame Labels

In [6]:
leadinglabel = labels.idxmax(axis=1).rename("framelabel").str.lower()

## Extract Frame Dimensions

In [7]:
# Build lookup index
dim_to_consider = dimensions.abs().idxmax(axis=1)
def get_dimension_direction(x):
    dim_pair = dim_to_consider[x.name]
    value = x[dim_pair]
    dim = dim_pair[0] if value > 0 else dim_pair[1]
    return dim
leadingdimension = dimensions.apply(get_dimension_direction, axis=1).rename("framedimension").str.lower()

## Extract Frame Structure

In [8]:
top_frames = structures[1].map(penman.decode).map(lambda x: x.top).value_counts()
selected_frames = top_frames[top_frames >= 200].drop(["and", "multi-sentence", "amr-error", "and_0", "thing"])
selected_frame_list = selected_frames.index.to_list()
selected_frames.to_dict(), len(selected_frames), sum(selected_frames)

({'say-01': 3423,
  'possible-01': 935,
  'report-01': 568,
  'cause-01': 453,
  'die-01': 451,
  'find-01': 369,
  'have-degree-91': 359,
  'watch-01': 333,
  'contrast-01': 330,
  'get-01': 326,
  'arrest-01': 312,
  'be-located-at-91': 278,
  'charge-05': 265,
  'open-01': 238,
  'show-01': 236,
  'kill-01': 233,
  'have-03': 230,
  'reveal-01': 230,
  'recommend-01': 227,
  'announce-01': 224,
  'want-01': 216,
  'close-01': 212,
  'win-01': 204},
 23,
 10652)

In [9]:
def find_first(sentence, words):
    def find(sentence, word):
        try:
            return sentence.index(word), word
        except ValueError:
            return float('inf'), None
    return min(find(sentence, word) for word in words)[1]

find_first_selected_frame = partial(find_first, words=selected_frame_list)
prevalentstructure = structures[1].map(find_first_selected_frame).fillna("other").rename("framestructure")
prevalentstructure.value_counts()

framestructure
other               29179
say-01               4007
have-degree-91       2583
cause-01             2379
possible-01          1929
win-01               1247
die-01               1032
report-01            1026
kill-01               770
have-03               711
get-01                675
find-01               650
watch-01              610
open-01               588
contrast-01           458
be-located-at-91      453
arrest-01             445
charge-05             428
want-01               424
close-01              418
recommend-01          360
show-01               331
reveal-01             299
announce-01           280
Name: count, dtype: int64

## Augment News

In [10]:
news_df = news_df.join(leadinglabel).join(leadingdimension).join(prevalentstructure)
news_df

,category,subcategory,title,abstract,link,title_entities,abstract_entities,framelabel,framedimension,framestructure
N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],morality,authority,other
N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...",economic,cheating,have-degree-91
N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",economic,subversion,other
N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ...",quality,cheating,other
N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...",capacity,degredation,say-01
...,...,...,...,...,...,...,...,...,...,...
N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...,https://assets.msn.com/labs/mind/BBWzQJK.html,"[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid...","[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid...",capacity,subversion,other
N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN,https://assets.msn.com/labs/mind/BBWzQYV.html,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...",[],quality,betrayal,say-01
N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[],morality,subversion,other
N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[],capacity,loyalty,win-01


In [11]:
news_df["category"].nunique(), news_df["framelabel"].nunique(), news_df["framedimension"].nunique(), news_df["framestructure"].nunique()

(17, 15, 10, 24)

In [12]:
def display_labels(label_set):
    print("'" + "', '".join(label_set) + "''")
    
display_labels(sorted(news_df["category"].unique()))
display_labels(sorted(news_df["framelabel"].unique()))
display_labels(sorted(news_df["framedimension"].unique()))
display_labels(sorted(news_df["framestructure"].unique()))

'autos', 'entertainment', 'finance', 'foodanddrink', 'health', 'kids', 'lifestyle', 'middleeast', 'movies', 'music', 'news', 'northamerica', 'sports', 'travel', 'tv', 'video', 'weather''
'capacity', 'crime', 'cultural', 'economic', 'external', 'fairness', 'health', 'legality,', 'morality', 'other', 'policy', 'political', 'public', 'quality', 'security''
'authority', 'betrayal', 'care', 'cheating', 'degredation', 'fairness', 'harm', 'loyalty', 'sanctity', 'subversion''
'announce-01', 'arrest-01', 'be-located-at-91', 'cause-01', 'charge-05', 'close-01', 'contrast-01', 'die-01', 'find-01', 'get-01', 'have-03', 'have-degree-91', 'kill-01', 'open-01', 'other', 'possible-01', 'recommend-01', 'report-01', 'reveal-01', 'say-01', 'show-01', 'want-01', 'watch-01', 'win-01''


# Augment Behavior

In [13]:
behavior_df = pd.read_csv("../input/mind-news-dataset/MINDsmall_train/behaviors.tsv", sep="\t", header=None, index_col=0, names=["user_id", "datetime", "history", "impression"]).dropna(subset="history")
behavior_df

,user_id,datetime,history,impression
1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...
...,...,...,...,...
156961,U21593,11/14/2019 10:24:05 PM,N7432 N58559 N1954 N43353 N14343 N13008 N28833...,N2235-0 N22975-0 N64037-0 N47652-0 N11378-0 N4...
156962,U10123,11/13/2019 6:57:04 AM,N9803 N104 N24462 N57318 N55743 N40526 N31726 ...,N3841-0 N61571-0 N58813-0 N28213-0 N4428-0 N25...
156963,U75630,11/14/2019 10:58:13 AM,N29898 N59704 N4408 N9803 N53644 N26103 N812 N...,N55913-0 N62318-0 N53515-0 N10960-0 N9135-0 N5...
156964,U44625,11/13/2019 2:57:02 PM,N4118 N47297 N3164 N43295 N6056 N38747 N42973 ...,N6219-0 N3663-0 N31147-0 N58363-0 N4107-0 N457...


In [14]:
# Check that users history never differs
def assert_history_same(x):
    assert (x["history"] == x["history"].iloc[0]).all()
behavior_df.groupby("user_id").progress_apply(assert_history_same)

100%|██████████| 49108/49108 [00:14<00:00, 3449.99it/s]


""


In [15]:
behavior_df["user_id"].nunique()

49108

In [16]:
def map_ids_to_label_sequence(news_ids, lookup_s):
    def id_to_label(x):
        # We split to deal with suffixes, such as click indicators of impressions
        parts = x.split("-")
        id_part = parts[0]
        suffix_parts = parts[1:]
        converted_id_part = lookup_s.loc[id_part]
        converted_with_suffx = "-".join([converted_id_part] + suffix_parts)
        return converted_with_suffx
    if not news_ids:
        return ""
    label_sequence = " ".join(map(id_to_label, news_ids))
    return label_sequence

In [17]:
behavior_df["category_history"] = behavior_df["history"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["category"].str.lower()))
behavior_df["framelabel_history"] = behavior_df["history"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["framelabel"].str.lower()))
behavior_df["framedimension_history"] = behavior_df["history"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["framedimension"].str.lower()))
behavior_df["framestructure_history"] = behavior_df["history"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["framestructure"].str.lower()))
behavior_df[behavior_df.columns[behavior_df.columns.str.endswith("history")]]

100%|██████████| 153727/153727 [01:02<00:00, 2468.75it/s]


,history,category_history,framelabel_history,framedimension_history,framestructure_history
1,N55189 N42782 N34694 N45794 N18445 N63302 N104...,tv sports tv news sports lifestyle movies news...,capacity capacity public crime economic morali...,degredation cheating betrayal authority subver...,other win-01 say-01 arrest-01 other other die-...
2,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,news news news finance travel news news news n...,crime economic quality economic quality crime ...,cheating degredation authority cheating harm b...,say-01 other find-01 other kill-01 say-01 die-...
3,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,lifestyle lifestyle news sports tv weather spo...,economic quality quality capacity capacity qua...,subversion betrayal degredation harm betrayal ...,cause-01 other other report-01 other other cau...
4,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,tv sports tv finance finance sports lifestyle ...,crime public crime economic economic crime eco...,cheating harm degredation subversion subversio...,arrest-01 other die-01 other have-degree-91 ar...
5,N10078 N56514 N14904 N33740,autos travel weather health,economic economic quality economic,fairness betrayal degredation care,other other have-degree-91 other
...,...,...,...,...,...
156961,N7432 N58559 N1954 N43353 N14343 N13008 N28833...,video news news news news finance video lifest...,quality crime political crime economic economi...,betrayal fairness harm betrayal betrayal harm ...,have-degree-91 say-01 other say-01 other other...
156962,N9803 N104 N24462 N57318 N55743 N40526 N31726 ...,finance sports sports news lifestyle music spo...,economic quality quality quality quality quali...,betrayal loyalty loyalty authority degredation...,have-degree-91 other other other show-01 other...
156963,N29898 N59704 N4408 N9803 N53644 N26103 N812 N...,travel news sports finance foodanddrink travel...,economic security capacity economic capacity c...,authority harm cheating betrayal subversion ca...,other say-01 be-located-at-91 have-degree-91 o...
156964,N4118 N47297 N3164 N43295 N6056 N38747 N42973 ...,lifestyle weather news finance entertainment t...,capacity capacity capacity economic capacity q...,subversion subversion authority authority degr...,other other other other other other find-01 ot...


In [18]:
behavior_df["category_impression"] = behavior_df["impression"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["category"].str.lower()))
behavior_df["framelabel_impression"] = behavior_df["impression"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["framelabel"].str.lower()))
behavior_df["framedimension_impression"] = behavior_df["impression"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["framedimension"].str.lower()))
behavior_df["framestructure_impression"] = behavior_df["impression"].str.split().progress_map(partial(map_ids_to_label_sequence, lookup_s=news_df["framestructure"].str.lower()))
behavior_df[behavior_df.columns[behavior_df.columns.str.endswith("impression")]]

100%|██████████| 153727/153727 [01:11<00:00, 2157.66it/s]


,impression,category_impression,framelabel_impression,framedimension_impression,framestructure_impression
1,N55689-1 N35729-0,sports-1 news-0,quality-1 economic-0,degredation-1 betrayal-0,die-01-1 kill-01-0
2,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,sports-0 news-0 autos-0 news-0 news-0 finance-...,capacity-0 political-0 economic-0 crime-0 qual...,degredation-0 cheating-0 subversion-0 degredat...,other-0 other-0 other-0 charge-05-0 say-01-0 o...
3,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,sports-0 news-0 finance-0 news-0 autos-0 sport...,capacity-0 crime-0 economic-0 capacity-0 econo...,cheating-0 degredation-0 subversion-0 fairness...,cause-01-0 say-01-0 other-0 kill-01-0 other-0 ...
4,N35729-0 N33632-0 N49685-1 N27581-0,news-0 sports-0 music-1 finance-0,economic-0 capacity-0 quality-1 political-0,betrayal-0 cheating-0 degredation-1 betrayal-0,kill-01-0 other-0 die-01-1 other-0
5,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,finance-0 finance-0 news-0 autos-1 finance-0 w...,economic-0 capacity-0 quality-0 economic-1 eco...,harm-0 fairness-0 degredation-0 degredation-1 ...,have-03-0 other-0 other-0 other-1 other-0 othe...
...,...,...,...,...,...
156961,N2235-0 N22975-0 N64037-0 N47652-0 N11378-0 N4...,sports-0 finance-0 news-0 sports-0 video-0 new...,capacity-0 economic-0 quality-0 security-0 qua...,sanctity-0 cheating-0 betrayal-0 loyalty-0 sub...,win-01-0 have-degree-91-0 other-0 other-0 watc...
156962,N3841-0 N61571-0 N58813-0 N28213-0 N4428-0 N25...,entertainment-0 foodanddrink-0 news-0 finance-...,quality-0 economic-0 economic-0 economic-0 mor...,betrayal-0 cheating-0 subversion-0 cheating-0 ...,other-0 other-0 other-0 have-degree-91-0 other...
156963,N55913-0 N62318-0 N53515-0 N10960-0 N9135-0 N5...,foodanddrink-0 tv-0 finance-0 foodanddrink-0 f...,capacity-0 capacity-0 economic-0 capacity-0 ec...,harm-0 harm-0 authority-0 betrayal-0 degredati...,be-located-at-91-0 cause-01-0 have-degree-91-0...
156964,N6219-0 N3663-0 N31147-0 N58363-0 N4107-0 N457...,autos-0 news-0 news-0 finance-0 news-0 movies-...,economic-0 crime-0 public-0 economic-0 politic...,sanctity-0 degredation-0 degredation-0 fairnes...,other-0 say-01-0 say-01-0 other-0 cause-01-0 s...


In [19]:
behavior_df.to_csv("behavior_label_sequences.csv")

In [20]:
!wc -l behavior_label_sequences.csv

153728 behavior_label_sequences.csv


In [21]:
pd.read_csv("behavior_label_sequences.csv", index_col=0)

,user_id,datetime,history,impression,category_history,framelabel_history,framedimension_history,framestructure_history,category_impression,framelabel_impression,framedimension_impression,framestructure_impression
1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,tv sports tv news sports lifestyle movies news...,capacity capacity public crime economic morali...,degredation cheating betrayal authority subver...,other win-01 say-01 arrest-01 other other die-...,sports-1 news-0,quality-1 economic-0,degredation-1 betrayal-0,die-01-1 kill-01-0
2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,news news news finance travel news news news n...,crime economic quality economic quality crime ...,cheating degredation authority cheating harm b...,say-01 other find-01 other kill-01 say-01 die-...,sports-0 news-0 autos-0 news-0 news-0 finance-...,capacity-0 political-0 economic-0 crime-0 qual...,degredation-0 cheating-0 subversion-0 degredat...,other-0 other-0 other-0 charge-05-0 say-01-0 o...
3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,lifestyle lifestyle news sports tv weather spo...,economic quality quality capacity capacity qua...,subversion betrayal degredation harm betrayal ...,cause-01 other other report-01 other other cau...,sports-0 news-0 finance-0 news-0 autos-0 sport...,capacity-0 crime-0 economic-0 capacity-0 econo...,cheating-0 degredation-0 subversion-0 fairness...,cause-01-0 say-01-0 other-0 kill-01-0 other-0 ...
4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,tv sports tv finance finance sports lifestyle ...,crime public crime economic economic crime eco...,cheating harm degredation subversion subversio...,arrest-01 other die-01 other have-degree-91 ar...,news-0 sports-0 music-1 finance-0,economic-0 capacity-0 quality-1 political-0,betrayal-0 cheating-0 degredation-1 betrayal-0,kill-01-0 other-0 die-01-1 other-0
5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,autos travel weather health,economic economic quality economic,fairness betrayal degredation care,other other have-degree-91 other,finance-0 finance-0 news-0 autos-1 finance-0 w...,economic-0 capacity-0 quality-0 economic-1 eco...,harm-0 fairness-0 degredation-0 degredation-1 ...,have-03-0 other-0 other-0 other-1 other-0 othe...
...,...,...,...,...,...,...,...,...,...,...,...,...
156961,U21593,11/14/2019 10:24:05 PM,N7432 N58559 N1954 N43353 N14343 N13008 N28833...,N2235-0 N22975-0 N64037-0 N47652-0 N11378-0 N4...,video news news news news finance video lifest...,quality crime political crime economic economi...,betrayal fairness harm betrayal betrayal harm ...,have-degree-91 say-01 other say-01 other other...,sports-0 finance-0 news-0 sports-0 video-0 new...,capacity-0 economic-0 quality-0 security-0 qua...,sanctity-0 cheating-0 betrayal-0 loyalty-0 sub...,win-01-0 have-degree-91-0 other-0 other-0 watc...
156962,U10123,11/13/2019 6:57:04 AM,N9803 N104 N24462 N57318 N55743 N40526 N31726 ...,N3841-0 N61571-0 N58813-0 N28213-0 N4428-0 N25...,finance sports sports news lifestyle music spo...,economic quality quality quality quality quali...,betrayal loyalty loyalty authority degredation...,have-degree-91 other other other show-01 other...,entertainment-0 foodanddrink-0 news-0 finance-...,quality-0 economic-0 economic-0 economic-0 mor...,betrayal-0 cheating-0 subversion-0 cheating-0 ...,other-0 other-0 other-0 have-degree-91-0 other...
156963,U75630,11/14/2019 10:58:13 AM,N29898 N59704 N4408 N9803 N53644 N26103 N812 N...,N55913-0 N62318-0 N53515-0 N10960-0 N9135-0 N5...,travel news sports finance foodanddrink travel...,economic security capacity economic capacity c...,authority harm cheating betrayal subversion ca...,other say-01 be-l